In [1]:
from prepross import *

In [2]:
config = Config()

In [3]:
config.Test_dataset_dir

'/home/mehrazin/Documents/DAV project/Repository/Music-Recommendation/Dataset/Test_sample_2.csv'

In [4]:
data = load_data(config)

In [7]:
data = create_session(data, config)

0    user_000001
       user_id                 timestamp            artist_name  \
0  user_000001 2006-08-13 13:59:20+00:00      Plaid & Bob Jaroc   
1  user_000001 2006-08-13 14:10:43+00:00      Plaid & Bob Jaroc   
2  user_000001 2006-08-13 14:19:06+00:00          Artful Dodger   
3  user_000001 2006-08-13 14:23:03+00:00           Rei Harakami   
4  user_000001 2006-08-13 14:55:14+00:00         Herbie Hancock   
5  user_000001 2006-08-13 14:59:59+00:00              Radiohead   
6  user_000001 2006-08-13 15:05:20+00:00                   高木正勝   
7  user_000001 2006-08-13 15:12:12+00:00  Black Jazz Chronicles   
8  user_000001 2006-08-13 15:17:35+00:00        Valerie Etienne   
9  user_000001 2006-08-13 15:36:22+00:00               Röyksopp   

                                 track_name Session  
0                 The Launching Of Big Face          
1  The Return Of Super Barrio - End Credits          
2                        What You Gonna Do?          
3                            

            user_id                 timestamp artist_name  \
110347  user_000008 2008-12-21 02:41:01+00:00  Kanye West   
110348  user_000008 2008-12-21 02:41:36+00:00  Kanye West   
110349  user_000008 2008-12-21 02:42:09+00:00  Kanye West   
110350  user_000008 2008-12-21 02:42:43+00:00  Kanye West   
110351  user_000008 2008-12-21 02:43:16+00:00  Kanye West   
110352  user_000008 2008-12-21 02:43:50+00:00  Kanye West   
110353  user_000008 2008-12-21 02:44:24+00:00  Kanye West   
110354  user_000008 2008-12-21 02:44:58+00:00  Kanye West   
110355  user_000008 2008-12-21 02:45:31+00:00  Kanye West   
110356  user_000008 2008-12-21 02:46:05+00:00  Kanye West   

                                             track_name Session  
110347                                        Heartless          
110348                      Amazing (Feat. Young Jeezy)          
110349                                    Love Lockdown          
110350                      Paranoid (Feat. Mr. Hudson)         

            user_id                 timestamp               artist_name  \
228301  user_000014 2006-10-26 17:50:31+00:00                  The Fray   
228302  user_000014 2006-10-26 17:54:50+00:00                  The Fray   
228303  user_000014 2006-10-27 15:26:32+00:00                 4 Strings   
228304  user_000014 2006-10-27 15:29:41+00:00                     Adema   
228305  user_000014 2006-10-27 15:34:11+00:00                     Adema   
228306  user_000014 2006-10-27 15:37:47+00:00                The Afters   
228307  user_000014 2006-10-27 15:41:49+00:00                The Afters   
228308  user_000014 2006-10-31 06:54:31+00:00                  Coldplay   
228309  user_000014 2006-10-31 15:58:45+00:00  Pretty Girls Make Graves   
228310  user_000014 2006-10-31 16:01:33+00:00               Ursula 1000   

                                             track_name Session  
228301                               How To Save A Life          
228302                                    

            user_id                 timestamp            artist_name  \
305701  user_000021 2005-05-24 01:32:59+00:00              Buzzcocks   
305702  user_000021 2005-05-24 01:37:41+00:00             Neil Young   
305703  user_000021 2005-05-24 01:40:44+00:00            Johnny Cash   
305704  user_000021 2005-05-24 01:42:09+00:00            Joey Ramone   
305705  user_000021 2005-05-24 01:44:44+00:00          Elliott Smith   
305706  user_000021 2005-05-24 01:49:43+00:00         New Bomb Turks   
305707  user_000021 2005-05-24 01:53:15+00:00  The New Pornographers   
305708  user_000021 2005-05-24 01:56:10+00:00              The Hives   
305709  user_000021 2005-05-24 01:58:43+00:00                The Jam   
305710  user_000021 2005-05-24 02:01:03+00:00     Bedouin Soundclash   

                                  track_name Session  
305701                              Lipstick          
305702                                  Ohio          
305703                                Bust

            user_id                 timestamp           artist_name  \
512673  user_000028 2005-11-19 21:26:18+00:00           Lloyd Banks   
512674  user_000028 2005-11-19 21:29:58+00:00             Tony Yayo   
512675  user_000028 2005-11-19 22:07:29+00:00                 Jay-Z   
512676  user_000028 2005-11-19 22:10:59+00:00                 Jay-Z   
512677  user_000028 2005-11-20 12:05:32+00:00  Asian Dub Foundation   
512678  user_000028 2005-11-20 12:41:34+00:00             Tony Yayo   
512679  user_000028 2005-11-20 13:12:26+00:00             Tony Yayo   
512680  user_000028 2005-11-20 15:53:49+00:00           Craig David   
512681  user_000028 2005-11-20 16:24:52+00:00          Wu-Tang Clan   
512682  user_000028 2005-11-20 16:46:13+00:00                Eminem   

                    track_name Session  
512673                 Get Low          
512674               Fake Love          
512675                 Lucifer          
512676             99 Problems          
512677       

...

962    user_000971
              user_id                 timestamp artist_name      track_name  \
13718168  user_000971 2005-12-09 02:34:00+00:00      Beyond          Effort   
13718169  user_000971 2005-12-09 02:36:01+00:00      Beyond        Feedback   
13718170  user_000971 2005-12-09 02:37:55+00:00      Beyond            Hoax   
13718171  user_000971 2005-12-09 02:42:33+00:00      Beyond   One Kind Word   
13718172  user_000971 2005-12-09 02:46:39+00:00      Beyond  Save Ourselves   
13718173  user_000971 2005-12-09 02:52:42+00:00      Beyond   Self Interest   
13718174  user_000971 2005-12-09 02:54:26+00:00      Beyond         Someday   
13718175  user_000971 2005-12-09 03:01:47+00:00      Beyond  Vampire Empire   
13718176  user_000971 2005-12-09 03:03:32+00:00      Beyond        Vitality   
13718177  user_000971 2005-12-09 03:08:44+00:00     Sublime            Hope   

         Session  
13718168          
13718169          
13718170          
13718171          
13718172        

969    user_000978
              user_id                 timestamp               artist_name  \
13803940  user_000978 2005-08-24 20:59:52+00:00                   Haggard   
13803941  user_000978 2005-08-24 21:06:42+00:00             Guns N' Roses   
13803942  user_000978 2005-08-24 21:35:22+00:00              Stratovarius   
13803943  user_000978 2005-08-24 21:57:13+00:00  The Alan Parsons Project   
13803944  user_000978 2005-08-24 22:00:43+00:00  The Alan Parsons Project   
13803945  user_000978 2005-08-25 09:58:39+00:00             Sarah Vaughan   
13803946  user_000978 2005-08-25 10:06:50+00:00               Hans Zimmer   
13803947  user_000978 2005-08-25 10:11:30+00:00                   Incubus   
13803948  user_000978 2005-08-25 10:56:38+00:00                 Semisonic   
13803949  user_000978 2005-08-25 11:30:45+00:00                       311   

                                   track_name Session  
13803940             In A Fullmoon Procession          
13803941             

975    user_000984
              user_id                 timestamp  \
13834914  user_000984 2005-07-02 08:39:23+00:00   
13834915  user_000984 2005-07-02 08:41:38+00:00   
13834916  user_000984 2005-07-02 08:46:41+00:00   
13834917  user_000984 2005-07-02 08:53:46+00:00   
13834918  user_000984 2005-07-02 08:57:33+00:00   
13834919  user_000984 2005-07-02 09:01:06+00:00   
13834920  user_000984 2005-07-02 19:08:08+00:00   
13834921  user_000984 2005-07-02 19:11:42+00:00   
13834922  user_000984 2005-07-03 07:51:40+00:00   
13834923  user_000984 2005-07-03 07:56:48+00:00   

                             artist_name                        track_name  \
13834914                   L'Arc~En~Ciel                             Honey   
13834915  Fela Ransome Kuti & Nigeria 70                        Witchcraft   
13834916              Christina Aguilera  Come On Over (All I Want Is You)   
13834917                    Mariah Carey                        Dreamlover   
13834918                     

981    user_000991
              user_id                 timestamp             artist_name  \
13886117  user_000991 2007-10-07 19:29:30+00:00                   Origa   
13886118  user_000991 2007-10-07 19:46:26+00:00  Snake River Conspiracy   
13886119  user_000991 2007-10-07 19:50:28+00:00  Snake River Conspiracy   
13886120  user_000991 2007-10-07 19:54:57+00:00  Snake River Conspiracy   
13886121  user_000991 2007-10-07 19:59:51+00:00  Snake River Conspiracy   
13886122  user_000991 2007-10-07 20:03:47+00:00  Snake River Conspiracy   
13886123  user_000991 2007-10-07 20:08:14+00:00  Snake River Conspiracy   
13886124  user_000991 2007-10-07 20:12:03+00:00  Snake River Conspiracy   
13886125  user_000991 2007-10-07 20:16:47+00:00  Snake River Conspiracy   
13886126  user_000991 2007-10-07 20:20:32+00:00  Snake River Conspiracy   

                  track_name Session  
13886117      Inner Universe          
13886118              Vulcan          
13886119               Breed          

988    user_000998
              user_id                 timestamp      artist_name  \
13974527  user_000998 2005-09-28 18:57:48+00:00   Arctic Monkeys   
13974528  user_000998 2005-09-28 18:59:36+00:00   Arctic Monkeys   
13974529  user_000998 2005-09-28 19:02:19+00:00   Arctic Monkeys   
13974530  user_000998 2005-09-28 19:06:57+00:00   Arctic Monkeys   
13974531  user_000998 2005-09-28 19:14:28+00:00      Arcade Fire   
13974532  user_000998 2005-09-28 19:19:15+00:00      Arcade Fire   
13974533  user_000998 2005-09-28 19:24:54+00:00  The Duke Spirit   
13974534  user_000998 2005-09-28 19:28:03+00:00  The Duke Spirit   
13974535  user_000998 2005-09-28 19:35:55+00:00   Porcupine Tree   
13974536  user_000998 2005-09-28 19:38:39+00:00   Porcupine Tree   

                                     track_name Session  
13974527  I Bet You Look Good On The Dancefloor          
13974528                         Space Invaders          
13974529                      A Certain Romance          


In [ ]:
se = list(df.Session)

In [ ]:
se = list(dict.fromkeys(se))

In [ ]:
print('  '.join(se))

In [ ]:
df[df.Session == se[999]]